# **📄 Document type classification baseline code**
> 문서 타입 분류 대회에 오신 여러분 환영합니다! 🎉     
> 아래 baseline에서는 ResNet 모델을 로드하여, 모델을 학습 및 예측 파일 생성하는 프로세스에 대해 알아보겠습니다.

## Contents
- Prepare Environments
- Import Library & Define Functions
- Hyper-parameters
- Load Data
- Train Model
- Inference & Save File


## 1. Prepare Environments

* 데이터 로드를 위한 구글 드라이브를 마운트합니다.
* 필요한 라이브러리를 설치합니다.

## 2. Import Library & Define Functions
* 학습 및 추론에 필요한 라이브러리를 로드합니다.
* 학습 및 추론에 필요한 함수와 클래스를 정의합니다.

In [2]:
import os
import time
import random

import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score

In [3]:
# 시드를 고정합니다.
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True

In [4]:
# 데이터셋 클래스를 정의합니다.
class ImageDataset(Dataset):
    def __init__(self, csv, path, transform=None):
        self.df = pd.read_csv(csv).values
        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img = np.array(Image.open(os.path.join(self.path, name)))
        if self.transform:
            img = self.transform(image=img)['image']
        return img, target

In [7]:
# one epoch 학습을 위한 함수입니다.
def train_one_epoch(loader, model, optimizer, loss_fn, device):
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = []

    pbar = tqdm(loader)
    for image, targets in pbar:
        image = image.to(device)
        targets = targets.to(device)

        model.zero_grad(set_to_none=True)

        preds = model(image)
        loss = loss_fn(preds, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        pbar.set_description(f"Loss: {loss.item():.4f}")

    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    ret = {
        "train_loss": train_loss,
        "train_acc": train_acc,
        "train_f1": train_f1,
    }

    return ret

## 3. Hyper-parameters
* 학습 및 추론에 필요한 하이퍼파라미터들을 정의합니다.

In [81]:
# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# data config
data_path = 'datasets_fin/'

# model config
model_name = 'resnext50_32x4d' # 'resnet50' 'efficientnet-b0', ...

# training config
img_size = 256
LR = 1.5e-3
EPOCHS = 30
BATCH_SIZE = 32
num_workers = 0

In [8]:
df = pd.read_csv("../data/train.csv")
print(df)

                        ID  target
0     002f99746285dfdd.jpg      16
1     008ccd231e1fea5d.jpg      10
2     008f5911bfda7695.jpg      10
3     009235e4c9c07af5.jpg       4
4     00b2f44967580c74.jpg      16
...                    ...     ...
1565  fed9e9ec4a77bc06.jpg       4
1566  feeade617aa68c45.jpg       7
1567  ff51dd281a8423f1.jpg      11
1568  ff8a6a251ce51c95.jpg       5
1569  ffc22136f958deb1.jpg       9

[1570 rows x 2 columns]


데이터 증강

In [9]:
import cv2
import pandas as pd
from tqdm import tqdm
import albumentations as A

# 원본 폴더와 증강 이미지 저장 폴더
input_dir = "../data/train"
output_dir = "../data/train_aug"
os.makedirs(output_dir, exist_ok=True)

# CSV 불러오기
df = pd.read_csv("../data/train.csv")

# 새 CSV 기록용
aug_data = []

# Albumentations transform 정의
augment = A.Compose([
    A.Rotate(limit=180, p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.GaussianBlur(p=0.5)
])

for idx, row in tqdm(df.iterrows(), total=len(df)):
    img_name = row['ID']
    label = row['target']
    img_path = os.path.join(input_dir, img_name)
    
    image = cv2.imread(img_path)
    
    # 원본 저장 및 CSV 기록
    cv2.imwrite(os.path.join(output_dir, img_name), image)
    aug_data.append({"filename": img_name, "label": label})

    # 증강 이미지 생성 (예: 2개)
    for i in range(4):
        augmented = augment(image=image)["image"]
        new_name = f"{os.path.splitext(img_name)[0]}_aug{i+1}.jpg"
        cv2.imwrite(os.path.join(output_dir, new_name), augmented)
        aug_data.append({"filename": new_name, "label": label})

# 새 CSV 저장
aug_df = pd.DataFrame(aug_data)
aug_df.to_csv("../data/aug_train.csv", index=False)

print("✅ 데이터 증강 및 CSV 생성 완료!")

100%|██████████| 1570/1570 [00:42<00:00, 37.12it/s]

✅ 데이터 증강 및 CSV 생성 완료!


## 4. Load Data
* 학습, 테스트 데이터셋과 로더를 정의합니다.

In [35]:
# augmentation을 위한 transform 코드
trn_transform = A.Compose([
    # 이미지 크기 조정
    A.Resize(height=img_size, width=img_size),
    A.Rotate(limit=180, p=0.5),  # ✅ Albumentations용 회전
    # 블러 (랜덤하게 흐림 적용)
    A.OneOf([
        A.MotionBlur(p=0.3),
        A.MedianBlur(blur_limit=3, p=0.3),
        A.GaussianBlur(blur_limit=(3,5), p=0.3),
    ], p=0.5),

    # 감마 보정 (밝기 대비 변화)
    A.RandomGamma(gamma_limit=(80, 120), p=0.5),

    # 랜덤 노이즈 (가우시안 노이즈 추가)
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.4),

    # 밝기, 대비 변화 (이미지 전반 조정)
    A.RandomBrightnessContrast(p=0.5),
    # images normalization
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    # numpy 이미지나 PIL 이미지를 PyTorch 텐서로 변환
    ToTensorV2(),
])

# test image 변환을 위한 transform 코드
tst_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

증강 미사용

In [ ]:
# Dataset 정의
trn_dataset = ImageDataset(
    "../data/train.csv",
    "../data/train/",
    transform=trn_transform
)

tst_dataset = ImageDataset(
    "../data/sample_submission.csv",
    "../data/test/",
    transform=tst_transform
)
print(len(trn_dataset), len(tst_dataset))

1570 3140


증강사용

In [78]:
# augmentation을 위한 transform 코드
trn_transform = A.Compose([
    # 이미지 크기 조정
    A.Resize(height=img_size, width=img_size),

    # images normalization
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    # numpy 이미지나 PIL 이미지를 PyTorch 텐서로 변환
    ToTensorV2(),
])

# test image 변환을 위한 transform 코드
tst_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

In [88]:
# Dataset 정의
trn_dataset = ImageDataset(
    "../data/aug_train.csv",
    "../data/train_aug/",
    transform=trn_transform
)
tst_dataset = ImageDataset(
    "../data/sample_submission.csv",
    "../data/test/",
    transform=tst_transform
)
print(len(trn_dataset), len(tst_dataset))

7850 3140


In [99]:
for _, target in trn_dataset:
    print(target)

16
16
16
16
16
10
10
10
10
10
10
10
10
10
10
4
4
4
4
4
16
16
16
16
16
5
5
5
5
5
15
15
15
15
15
16
16
16
16
16
15
15
15
15
15
14
14
14
14
14
5
5
5
5
5
9
9
9
9
9
15
15
15
15
15
15
15
15
15
15
13
13
13
13
13
7
7
7
7
7
11
11
11
11
11
10
10
10
10
10
11
11
11
11
11
15
15
15
15
15
7
7
7
7
7
10
10
10
10
10
11
11
11
11
11
15
15
15
15
15
7
7
7
7
7
5
5
5
5
5
10
10
10
10
10
2
2
2
2
2
9
9
9
9
9
14
14
14
14
14
15
15
15
15
15
5
5
5
5
5
5
5
5
5
5
16
16
16
16
16
8
8
8
8
8
14
14
14
14
14
10
10
10
10
10
12
12
12
12
12
11
11
11
11
11
3
3
3
3
3
16
16
16
16
16
16
16
16
16
16
2
2
2
2
2
11
11
11
11
11
7
7
7
7
7
16
16
16
16
16
0
0
0
0
0
3
3
3
3
3
7
7
7
7
7
12
12
12
12
12
15
15
15
15
15
12
12
12
12
12
16
16
16
16
16
10
10
10
10
10
15
15
15
15
15
13
13
13
13
13
13
13
13
13
13
0
0
0
0
0
16
16
16
16
16
3
3
3
3
3
8
8
8
8
8
1
1
1
1
1
5
5
5
5
5
7
7
7
7
7
9
9
9
9
9
10
10
10
10
10
1
1
1
1
1
11
11
11
11
11
0
0
0
0
0
0
0
0
0
0
15
15
15
15
15
3
3
3
3
3
8
8
8
8
8
10
10
10
10
10
1
1
1
1
1
3
3
3
3
3
9
9
9
9
9
10
10
10
10
10


KeyboardInterrupt: 

In [80]:
# DataLoader 정의
trn_loader = DataLoader(
    trn_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True,
    drop_last=False
)
tst_loader = DataLoader(
    tst_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=True
)

## 5. Train Model
* 모델을 로드하고, 학습을 진행합니다.

In [82]:
# load model
model = timm.create_model(
    model_name,
    pretrained=True,
    num_classes=17
).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)


In [ ]:
EPOCHS = 30
early_stopping_patience = 5  # 5 epoch 동안 F1 향상이 없으면 학습 중단
for epoch in range(EPOCHS):
    ret = train_one_epoch(trn_loader, model, optimizer, loss_fn, device=device)
    ret['epoch'] = epoch

    log = ""
    for k, v in ret.items():
      log += f"{k}: {v:.4f}\n"
    print(log)

Loss: 0.0669: 100%|██████████| 123/123 [00:47<00:00,  2.56it/s]


train_loss: 0.0186
train_acc: 0.9940
train_f1: 0.9941
epoch: 0.0000



Loss: 0.0042: 100%|██████████| 123/123 [00:48<00:00,  2.56it/s]


train_loss: 0.0221
train_acc: 0.9921
train_f1: 0.9917
epoch: 1.0000



Loss: 0.0145: 100%|██████████| 123/123 [00:48<00:00,  2.56it/s]


train_loss: 0.0294
train_acc: 0.9899
train_f1: 0.9884
epoch: 2.0000



Loss: 0.0011: 100%|██████████| 123/123 [00:47<00:00,  2.57it/s]


train_loss: 0.0296
train_acc: 0.9904
train_f1: 0.9897
epoch: 3.0000



Loss: 0.0066: 100%|██████████| 123/123 [00:47<00:00,  2.56it/s]

train_loss: 0.0214
train_acc: 0.9934
train_f1: 0.9935
epoch: 4.0000



K-fold

In [13]:
from sklearn.metrics import f1_score
import torch

def validate(val_loader, model, device='cuda'):
    model.eval()  # 평가 모드
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            preds = torch.argmax(outputs, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Macro F1 score 계산
    f1 = f1_score(all_labels, all_preds, average='macro')
    return f1

In [83]:
# load model
model = timm.create_model(
    model_name,
    pretrained=True,
    num_classes=17
).to(device)
loss_fn = nn.CrossEntropyLoss()
#optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3)

In [15]:
def validate(val_loader, model, device='cuda'):
    model.eval()  # 평가 모드
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            preds = torch.argmax(outputs, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Macro F1 score 계산
    f1 = f1_score(all_labels, all_preds, average='macro')
    return f1

In [61]:
EPOCHS = 30
best_val_f1 = 0.0
patience = 5  # F1 향상이 없으면 몇 epoch 기다릴지
counter = 0

for epoch in range(EPOCHS):
    #학습
    ret = train_one_epoch(trn_loader, model, optimizer, loss_fn, device=device)
    
    #로그 출력
    log = ""
    for k, v in ret.items():
        log += f"{k}: {v:.4f}\n"
    print(log)
    
    #Early Stopping 체크
    if ret['train_f1'] > best_val_f1:
        best_val_f1 = ret['train_f1']
        counter = 0
        # 가장 좋은 모델 저장
        torch.save(model.state_dict(), "best_model.pth")
    else:
        counter += 1
        if counter >= patience:
            print(f"train F1 향상이 {patience} epoch 동안 없어서 학습 종료")
            break

    # 스케줄러 step (metric 기반)
    scheduler.step(ret['train_f1'])

Loss: 0.2268: 100%|██████████| 123/123 [00:56<00:00,  2.17it/s]


train_loss: 0.8412
train_acc: 0.7552
train_f1: 0.7443



Loss: 0.0833: 100%|██████████| 123/123 [00:53<00:00,  2.30it/s]


train_loss: 0.1201
train_acc: 0.9597
train_f1: 0.9573



Loss: 0.0157: 100%|██████████| 123/123 [00:53<00:00,  2.30it/s]


...

train_loss: 0.0188
train_acc: 0.9934
train_f1: 0.9934



Loss: 0.0171: 100%|██████████| 123/123 [00:53<00:00,  2.29it/s]


train_loss: 0.0176
train_acc: 0.9939
train_f1: 0.9937



Loss: 0.0446: 100%|██████████| 123/123 [00:53<00:00,  2.29it/s]

train_loss: 0.0108
train_acc: 0.9971
train_f1: 0.9966

train F1 향상이 5 epoch 동안 없어서 학습 종료


In [84]:
from sklearn.model_selection import StratifiedKFold
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset

# 예시 하이퍼파라미터
EPOCHS = 100
patience = 10
k_folds = 5

In [85]:
import torch
from sklearn.metrics import f1_score
from tqdm import tqdm

def evaluate(val_loader, model, device):
    """
    Validation 데이터셋으로 F1 스코어 계산
    Args:
        val_loader : DataLoader
        model      : PyTorch 모델
        device     : 'cuda' 또는 'cpu'
    Returns:
        val_f1    : float, macro F1 score
    """
    model.eval()  # 평가 모드
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="Evaluating", leave=False):
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            preds = outputs.argmax(dim=1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    # macro F1 계산
    val_f1 = f1_score(all_labels, all_preds, average='macro')
    return val_f1

In [ ]:
# Stratified KFold로 데이터 나누기
skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

# dataset.targets 가 레이블 배열
# Dataset에서 labels 추출
labels = np.array([trn_dataset[i][1] for i in range(len(trn_dataset))])

for fold, (train_idx, val_idx) in enumerate(skf.split(np.arange(len(trn_dataset)), labels)):
    print(f"\nFold {fold+1}/{k_folds}")

    # Subset 생성
    train_subset = Subset(trn_dataset, train_idx)
    val_subset = Subset(trn_dataset, val_idx)
    
    trn_loader = DataLoader(train_subset, batch_size=BATCH_SIZE, shuffle=True,
                            num_workers=num_workers, pin_memory=True, drop_last=False)
    val_loader = DataLoader(val_subset, batch_size=BATCH_SIZE, shuffle=False,
                            num_workers=num_workers, pin_memory=True, drop_last=False)
    # 모델 초기화
    model = timm.create_model(model_name, pretrained=True, num_classes=17).to(device)

    # 옵티마이저 / 스케줄러
    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3)

    best_val_f1 = 0.0
    counter = 0

    for epoch in range(EPOCHS):
        # train 한 epoch 수행
        ret = train_one_epoch(trn_loader, model, optimizer, loss_fn, device=device)

        # validation 평가
        val_f1 = evaluate(val_loader, model, device=device)

        print(f"Epoch {epoch+1}: train F1 {ret['train_f1']:.4f} | val F1 {val_f1:.4f}")

        # Early stopping
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            counter = 0
            torch.save(model.state_dict(), f"best_model_fold{fold+1}.pth")
        else:
            counter += 1
            if counter >= patience:
                print(f"Validation F1 향상이 {patience} epoch 동안 없어서 학습 종료")
                break

        # scheduler step
        scheduler.step(val_f1)

AttributeError: 'ImageDataset' object has no attribute 'targets'

In [ ]:
import torch
import numpy as np
from tqdm import tqdm

In [ ]:
# fold별 예측 저장 리스트
all_preds = []

for fold in range(k_folds):
    # 모델 초기화
    model = timm.create_model(model_name, pretrained=False, num_classes=17).to(device)
    
    # 저장된 best 모델 가중치 로드
    model.load_state_dict(torch.load(f"best_model_fold{fold+1}.pth", map_location=device))
    model.eval()
    
    preds_list = []
    with torch.no_grad():
        for images, _ in tqdm(tst_loader, desc=f"Predicting Fold {fold+1}"):
            images = images.to(device)
            preds = model(images)
            preds = torch.softmax(preds, dim=1)  # 확률값으로 변환
            preds_list.append(preds.cpu().numpy())
    
    # fold별 예측값 합치기
    all_preds.append(np.concatenate(preds_list, axis=0))

# numpy로 변환: shape (fold, N, num_classes)
all_preds = np.stack(all_preds)

# 확률 평균 앙상블
final_preds = all_preds.mean(axis=0)  # (N, num_classes)

# 최종 클래스 예측
final_labels = final_preds.argmax(axis=1)

print("예측 완료! 결과 shape:", final_labels.shape)

In [ ]:
pred_df = pd.DataFrame(tst_dataset.df, columns=['ID', 'target'])
pred_df['target'] = final_labels
pred_df.to_csv('submission.csv', index=False)

print("✅ submission.csv 저장 완료!")

# 6. Inference & Save File
* 테스트 이미지에 대한 추론을 진행하고, 결과 파일을 저장합니다.

In [69]:
model = timm.create_model("convnext_tiny", pretrained=True, num_classes=17).to(device)
model.load_state_dict(torch.load("best_model.pth"))
model.eval()

ConvNeXt(
  (stem): Sequential(
    (0): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
    (1): LayerNorm2d((96,), eps=1e-06, elementwise_affine=True)
  )
  (stages): Sequential(
    (0): ConvNeXtStage(
      (downsample): Identity()
      (blocks): Sequential(
        (0): ConvNeXtBlock(
          (conv_dw): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
          (norm): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=96, out_features=384, bias=True)
            (act): GELU()
            (drop1): Dropout(p=0.0, inplace=False)
            (norm): Identity()
            (fc2): Linear(in_features=384, out_features=96, bias=True)
            (drop2): Dropout(p=0.0, inplace=False)
          )
          (shortcut): Identity()
          (drop_path): Identity()
        )
        (1): ConvNeXtBlock(
          (conv_dw): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)


In [70]:
preds_list = []

model.eval()
for image, _ in tqdm(tst_loader):
    image = image.to(device)

    with torch.no_grad():
        preds = model(image)
    preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())

100%|██████████| 99/99 [00:16<00:00,  6.14it/s]


In [71]:
pred_df = pd.DataFrame(tst_dataset.df, columns=['ID', 'target'])
pred_df['target'] = preds_list

In [72]:
sample_submission_df = pd.read_csv("../data/sample_submission.csv")
assert (sample_submission_df['ID'] == pred_df['ID']).all()

In [73]:
pred_df.to_csv("pred.csv", index=False)

In [74]:
pred_df.head()

,ID,target
0,0008fdb22ddce0ce.jpg,16
1,00091bffdffd83de.jpg,12
2,00396fbc1f6cc21d.jpg,5
3,00471f8038d9c4b6.jpg,12
4,00901f504008d884.jpg,2


# 7. efficientnet_b2

In [48]:
# load model
model = timm.create_model(
    "efficientnet_b1",
    pretrained=True,
    num_classes=17
).to(device)

loss_fn = nn.CrossEntropyLoss()
#optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3)

In [ ]:
EPOCHS = 30
best_val_f1 = 0.0
patience = 10  # F1 향상이 없으면 몇 epoch 기다릴지
counter = 0

for epoch in range(EPOCHS):
    #학습
    ret = train_one_epoch(trn_loader, model, optimizer, loss_fn, device=device)
    
    #로그 출력
    log = ""
    for k, v in ret.items():
        log += f"{k}: {v:.4f}\n"
    print(log)
    
    #Early Stopping 체크
    if ret['train_f1'] > best_val_f1:
        best_val_f1 = ret['train_f1']
        counter = 0
        # 가장 좋은 모델 저장
        torch.save(model.state_dict(), "best_model.pth")
    else:
        counter += 1
        if counter >= patience:
            print(f"train F1 향상이 {patience} epoch 동안 없어서 학습 종료")
            break

    # 스케줄러 step (metric 기반)
    scheduler.step(ret['train_f1'])

Loss: 0.1420: 100%|██████████| 50/50 [00:13<00:00,  3.77it/s]


train_loss: 0.0320
train_acc: 0.9892
train_f1: 0.9884



Loss: 0.7674: 100%|██████████| 50/50 [00:13<00:00,  3.79it/s]


train_loss: 0.0359
train_acc: 0.9917
train_f1: 0.9911



Loss: 0.5820: 100%|██████████| 50/50 [00:13<00:00,  3.78it/s]


...

train_loss: 0.0666
train_acc: 0.9955
train_f1: 0.9958



Loss: 0.0082: 100%|██████████| 50/50 [00:13<00:00,  3.80it/s]


train_loss: 0.0107
train_acc: 0.9975
train_f1: 0.9976



Loss: 0.8638: 100%|██████████| 50/50 [00:13<00:00,  3.81it/s]

train_loss: 0.0242
train_acc: 0.9968
train_f1: 0.9967

train F1 향상이 5 epoch 동안 없어서 학습 종료


# convnext_tiny

In [58]:
# load model
model = timm.create_model(
    "convnext_tiny",
    pretrained=True,
    num_classes=17
).to(device)

loss_fn = nn.CrossEntropyLoss()
#optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3)

model.safetensors:   0%|          | 0.00/114M [00:00<?, ?B/s]

In [ ]:
EPOCHS = 40
best_val_f1 = 0.0
patience = 10  # F1 향상이 없으면 몇 epoch 기다릴지
counter = 0

In [68]:
EPOCHS = 60
counter = 0

for epoch in range(EPOCHS):
    #학습
    ret = train_one_epoch(trn_loader, model, optimizer, loss_fn, device=device)
    
    #로그 출력
    log = ""
    for k, v in ret.items():
        log += f"{k}: {v:.4f}\n"
    print(log)
    
    #Early Stopping 체크
    if ret['train_f1'] > best_val_f1:
        best_val_f1 = ret['train_f1']
        counter = 0
        # 가장 좋은 모델 저장
        torch.save(model.state_dict(), "best_model.pth")
    else:
        counter += 1
        if counter >= patience:
            print(f"train F1 향상이 {patience} epoch 동안 없어서 학습 종료")
            #break

    # 스케줄러 step (metric 기반)
    scheduler.step(ret['train_f1'])

Loss: 0.0127: 100%|██████████| 50/50 [00:17<00:00,  2.84it/s]


train_loss: 0.0940
train_acc: 0.9592
train_f1: 0.9530

train F1 향상이 10 epoch 동안 없어서 학습 종료


Loss: 0.0278: 100%|██████████| 50/50 [00:17<00:00,  2.88it/s]


train_loss: 0.0945
train_acc: 0.9637
train_f1: 0.9621

train F1 향상이 10 epoch 동안 없어서 학습 종료


Loss: 0.0000: 100%|██████████| 50/50 [00:17<00:00,  2.83it/s]


...

train_loss: 0.0969
train_acc: 0.9580
train_f1: 0.9558

train F1 향상이 10 epoch 동안 없어서 학습 종료


Loss: 0.0000: 100%|██████████| 50/50 [00:17<00:00,  2.89it/s]


train_loss: 0.0938
train_acc: 0.9599
train_f1: 0.9571

train F1 향상이 10 epoch 동안 없어서 학습 종료


Loss: 0.0000: 100%|██████████| 50/50 [00:17<00:00,  2.85it/s]

train_loss: 0.0850
train_acc: 0.9605
train_f1: 0.9585

train F1 향상이 10 epoch 동안 없어서 학습 종료
